# The basics of awkward arrays

At the front and formost of coffea is a completely new syntax for expressing analysis
computations: `awkward arrays` and it's index based notation. For people coming from a
more traditional loop-based programming syntax, the syntax will take some getting use 
to, but this tutorial can hopefully help you understand how to understand the syntax 
and how to understand the various method.

Let use begin by first understanding what you need to explore the contents of a typical
ntuple file using coffea related tools. First you can download the dummy ntuples files and
 the corresponding schema files from the main repository to your working directory in 
 your terminal:

```sh 
cd   <WORKINGDIRECTORY>
wget https://raw.githubusercontent.com/UMDCMS/CoffeaTutorial/main/samples/dummy_nanoevents.root 
wget https://raw.githubusercontent.com/UMDCMS/CoffeaTutorial/main/samples/dummyschema.py
```

We can use the usual ROOT tools to look at the contents of the `dummy_nanoevent.root` 
file. But let us focus on using coffea tools alone. 

First import the relevent coffea objects: 



In [1]:
from coffea.nanoevents import NanoEventsFactory
from dummyschema import DummySchema 
import numpy as np
import awkward1 as ak

Now we can create the event list as an awkward array using coffea tools like:


In [2]:
events = NanoEventsFactory.from_root( 'file:dummy_nanoevents.root', # The file, notice the prefix `file:` for local file operation
                                    'Events',                       # Name of the tree object to open   
                                    entry_stop=50,                  # Limit the number of events to process, nice for small scale debugging
                                    schemaclass=DummySchema
).events() 

The last `schemaclass` argument will be let unexplained for now, see the schema tutorials to 
learn more about what this is. Here we have created the events as a awkward array. To see that 
is stored in the array we can use:


In [3]:
print(events.fields)

['Electron', 'Muon', 'Jet']


Indicating the collections that are stored in the awkward array. To see how many events exists in in our file
we can use the typical python method:


In [4]:
print(len(events))

50


The 50 here in corresponds correctly to the `entry_stop` used in to open the file. Next we can, of course, start to explore the contents of the various object collections. One can acess the fields of the event as if it was a regular data memeber 

In [5]:
print(events.Electron.fields)
print(events.Jet.fields)

['pt', 'eta', 'phi', 'mass', 'charge', 'flag', 'dxy', 'dxyErr', 'dz', 'dzErr']
['pt', 'eta', 'phi', 'energy', 'ID', 'SubjetsCounts', 'Subjets']


Ah ha! We we are starting to see numbers we can play around with. Notice that coffea was written with High energy physics analysis in mind, so even if the electron energy doesn't look like it is stored from the output of the fields, we can still access methods that we will typically associate with 4-vectors. In particular, notice that we can call the `energy` field of the electron collection, even though the energy field isn't explicitly defined. Coffea is designed with 4 vectors in mind, so the energy collection is calculated on the fly. 

In [6]:
print(events.Electron.pt)
print(events.Electron.energy)

[[49.3, 38.1], [48.9, 43.9, 50.8, 43.9], ... [48.9, 42.9, 41.7, 56.3, 60.3, 35.5]]
[[52.6, 41.8], [49.5, 45.1, 56.6, 44.1], ... [56.8, 44.9, 55.5, 56.7, 82.5, 35.6]]


Now, looking at the output, we can begin to get a grasp of what awkward arrays are: the variable `events.Electron.pt` variable  represents a N events times A objects array of floating point, the `events.Electon` variable represents and N events times A objects time K fields of *collection* of floating point arrays, and the `events` variable reprents ths N times a certain set of collections (in this case three collections: `['Electron', 'Muon', and 'Jet']`) is recorded. 

The "awkward" part of the array refers to two parts, first the value of `A` is differnt for each event and for each collection. In this demonstration, our first event has 2 electrons, the second event has 4 electron and so one. The second part of each collection can have a different number of fields. In a sense, the `events`, `Electron` and `pt` variables are just a easy way for represting the final `NxA` array that we might be intested in for the analysis. In our case the `N` number of events is whatis called as the outer most **dimension** or axis of the various objects, `A` is the one inner dimesion of the of array. `K` is not a true dimesion in the sense it can be though of a book keeping object used to keep track of how many awkward arrays are present, so in this sense, we can say the the `events.Electron` is a `NxA` object/collection array, as opposed to the `events.Electron.pt` being a `NxA` data array.



We can use the usual index notation to look at a particular object of interest. For instance if we want to look at the 0-th electron of the 1-st event in our event list, we can write:

In [7]:
print(events[1].Electron[0])

... dzErr: 0.0284, eta: 0.149, flag: 3, mass: 0.00051, phi: 0.997, pt: 48.9}


But the real power of using awkward arrays is for using awkward arrays comes in when you don't explicily use a concrete index, and instead call calculation all in an abstract form

## Basic object and event selection 

Let us start with the most basic example of event selection. Say we want to select event with electrons that have $p_T > 50$ GeV and $|\eta| < 0.5$. The awkward array allows us to write something like:

In [8]:
mask_pt = events.Electron.pt > 50
mask_eta = np.abs(events.Electron.eta) < 0.5
ele_mask = mask_pt & mask_eta
print(mask_pt)
print(mask_eta)
print(ele_mask)

[[False, False], [False, False, True, ... [False, False, False, True, True, False]]
[[True, True], [True, True, True, True, ... [False, True, False, True, False, True]]
[[False, False], [False, False, True, ... [False, False, False, True, False, False]]


We can see that the usual logic comparision operators generate a `NxA` boolean array telling use which electron (or more specifically which electron.pt and electron etas) pass this particular selection criteia. This particular boolean array generated from a logic operation on usual arrays is typically call a `mask`. We can use the typical boolean operation `&` operation to get the intersect of multiple masks, or maybe the `|` operator for the union. Now the problem is where can we use this mask? The answer is any array that has a `NxA` structure and recieve these masks to create a reduced array!

In [9]:
print(events.Electron.pt[ele_mask])
print(events.Electron.eta[ele_mask])
selectedElectrons = events.Electron[ele_mask]
print(selectedElectrons.pt)

[[], [50.8], [68.5], [], [61], [54.7, ... 54.6, 58.9], [54.2], [], [54.5], [56.3]]
[[], [0.472], [0.0729], [], [0.0954], ... 0.136], [0.155], [], [0.411], [0.125]]
[[], [50.8], [68.5], [], [61], [54.7, ... 54.6, 58.9], [54.2], [], [54.5], [56.3]]


Probably the most important place to put the mask is the directly in the `events.Electron` index, this generates a new collection of electrons that preserves the `NxA` structure, but have verious collection instances filterd out. If you are familiar with `numpy`, this sort if index-based array filtering look familiar. The difference is that because awkward arrays accecpt arrays of varying inner dimensions, it can truely preserve the structure of such selection, rather than having everything be flattend out. 

In [10]:
x = np.array([1,2,3,4,5,6,7,8,1,1,1,2])
print( x[x% 2 == 0])
y = np.array([[1,2,3,4],[5,6,7,8],[1,1,1,2]])
print( y[y%2==0])
z = ak.Array([[1,2,3,4],[5,6,7,8],[1,1,1,2]])
print(z[z%2==0])

[2 4 6 8 2]
[2 4 6 8 2]
[[2, 4], [6, 8], [2]]


Now suppose we only want events that have at least 1 electron selected event. What we need are a set of functions that can reduces this `NxA'` array to something of just dimesion `N`. Formally this is called **reduction** operations, and the awkward package has a large set of functions that can reduce the dimension of arrays. In our case, what we want is:

In [11]:
electron_count = ak.count(selectedElectrons.pt, axis=-1)
event_mask = electron_count >= 1
print(event_mask.__repr__)

<bound method Array.__repr__ of <Array [False, True, True, ... True, True] type='50 * bool'>>


To break this down, `ak.count`, as the method name suggests "counts" the number of elements along a certain axis, in our case, what we are intersted is the inner most dimension/axis, hence the typical python notation of `axis=-1`. Using this we can run the event selection using the usual masking notation:

In [12]:
selectedEvents = events[event_mask]
print(event_mask)
print(events.Electron.pt)
print(selectedEvents.Electron.pt)
print(len(selectedEvents))

[False, True, True, False, True, True, ... False, True, True, False, True, True]
[[49.3, 38.1], [48.9, 43.9, 50.8, 43.9], ... [48.9, 42.9, 41.7, 56.3, 60.3, 35.5]]
[[48.9, 43.9, 50.8, 43.9], [46.8, 56.4, ... [48.9, 42.9, 41.7, 56.3, 60.3, 35.5]]
30


Here we can confirm that the first event to pass the event selection is the 1-st event in the event list, and the 0-th instance in the  `selectedEvents.Electron.pt` result of the selectedEvents indeed corresponds to the values stored in the 1-st event of the orignal event list. 

## Object storce and collection creation

Having completed the selection, we might be rather annoyed that we didn't just store the selected Electron, since these are the objects that we are likely going to use for further calculation. Following from the code above, what we can do is add the additional selection to the `selectedElectrons` collections. This is valid since the `N` dimesional event mask "makes sense" performed on the `NxA'` dimesional selectedElectrons object.


In [13]:
our_selectedElectrons = selectedElectrons[event_mask]
print(our_selectedElectrons.pt)
print(len(our_selectedElectrons))

[[50.8], [68.5], [61], [54.7], [59, ... [50.9, 54.6, 58.9], [54.2], [54.5], [56.3]]
30


However, this is rather undesirable, since now we have some a whole bunch of detected collections, and event lists that we need to take care of: `selectedElectrons`, `selectedEvents`, `out_selectedEvents`. And this is with just one toy object selection. One can imagine if there isn't some sort of way to store collections into events, the analysis code will get out of hands very quick. This also ties into the topic that there might be certain physics quantities that are specific to a certain analysis that would might be used for the analysis object selection and would be nice to add to the electron collection if it isn't a standard variable that is maintained by the NanoAOD development team. Here we are going to add a very artificial example of calculating the inverse of the electron pt, then selecting on the inverse pt.  This very simple example will demonstrate the typical syntax used for storing variables as well as exposing one of the parculiar quirks of awkward arrays:

In [14]:
print('First attempt at adding extended variables to events')
events.Electron['invpt'] = 1/events.Electron.pt
events['selectedElectron_1'] = events.Electron[events.Electron.pt > 50]

print(events.fields)
print(events.Electron.fields)
print(events.selectedElectron_1.fields)

print('\n\nSecond attemp at adding extended variables to events')
events['myElectron'] = events.Electron[:]
events.myElectron['invpt'] = 1/events.myElectron.pt
events['selectedElectron_2'] = events.myElectron[events.myElectron.pt > 50]

print(events.fields)
print(events.myElectron.fields)
print(events.selectedElectron_2.fields)

print('\n\nThird attemp at adding extended variables to events')
myElectron = events.Electron[:]
myElectron['invpt'] = 1/myElectron.pt
events['selectedElectron_3'] = myElectron[myElectron.pt > 50]

print(events.fields)
print(myElectron.fields)
print(events.selectedElectron_3.fields)


First attempt at adding extended variables to events
['Electron', 'Muon', 'Jet', 'selectedElectron_1']
['charge', 'dxy', 'dxyErr', 'dz', 'dzErr', 'eta', 'flag', 'mass', 'phi', 'pt']
['charge', 'dxy', 'dxyErr', 'dz', 'dzErr', 'eta', 'flag', 'mass', 'phi', 'pt']


Second attemp at adding extended variables to events
['Electron', 'Muon', 'Jet', 'selectedElectron_1', 'myElectron', 'selectedElectron_2']
['charge', 'dxy', 'dxyErr', 'dz', 'dzErr', 'eta', 'flag', 'mass', 'phi', 'pt']
['charge', 'dxy', 'dxyErr', 'dz', 'dzErr', 'eta', 'flag', 'mass', 'phi', 'pt']


Third attemp at adding extended variables to events
['Electron', 'Muon', 'Jet', 'selectedElectron_1', 'myElectron', 'selectedElectron_2', 'selectedElectron_3']
['charge', 'dxy', 'dxyErr', 'dz', 'dzErr', 'eta', 'flag', 'mass', 'phi', 'pt', 'invpt']
['charge', 'dxy', 'dxyErr', 'dz', 'dzErr', 'eta', 'flag', 'mass', 'phi', 'pt', 'invpt']



Lets get the straightforward part of the code clear up. The addition of collections looks very straight forward, one can think of the `events` as something that looks like a "dictionary of collection with a common outer dimension", so the addition of the two electron collections to the event has a very distionary-esque notation. What is strange is the persistence of the extended collection for the electrons. Logically, the operation looks identical, but the first attempt to add the new variable `invpt` directly to `events.Electron` fails to persist, and thus all direct extensions of `events.Electron` doesn't include the new `invpt` field. 

The reason for this is rather technical regarding the mutability of objects in python and awkward. The rule-of-thumb is that collections that are directly generated from the file, (a.k.a. the collections directly obtained listed the `events.fields` immediate after opening a file) can **never** be altered, and therefore cannot have extended variables added to them. To create an extended variable to some collection, we will need to make some sort of copy of the original either by some trivial kinematic selection (ex. `myElectrons = events.Electrons[events.Electrons.pt > 0]`) or some trivial splicing (`myElectrons = events.Electrons[:]`). Another feature of mutability is that once the collection is added to the event collection, it becomes immutable. That is why the third attempt is the one that adds the both the electron extended variable and the extednded electron collection to the event.

Because of these quirks, it would typically be worth it to wrap the object selection into a function if the object selection is typical within an analysis, and it also helps with code readability

In [15]:
def SelectElectrons(electron):
    electron = electron[electron.pt > 50]
    electron['invpt']  = 1.0 / electron.pt
    return electron

events['selectedElectron_f'] = SelectElectrons(events.Electron)
print(events.fields)
print(events.selectedElectron_f.fields)

['Electron', 'Muon', 'Jet', 'selectedElectron_1', 'myElectron', 'selectedElectron_2', 'selectedElectron_3', 'selectedElectron_f']
['charge', 'dxy', 'dxyErr', 'dz', 'dzErr', 'eta', 'flag', 'mass', 'phi', 'pt', 'invpt']


Once the new object collection has been added to the event collection, they will persist to arbitrary levels of event selection:


In [16]:
myevents = events[ak.count(events.selectedElectron_f.pt,axis=-1) > 0 ]

print(myevents.fields)
print(myevents.selectedElectron_f.fields)

myevents = events[ak.count(events.selectedElectron_f.pt,axis=-1) > 1 ]

print(myevents.fields)
print(myevents.selectedElectron_f.fields)
myevents = events[ak.count(events.selectedElectron_f.pt,axis=-1) > 2 ]

print(myevents.fields)
print(myevents.selectedElectron_f.fields)


['Electron', 'Muon', 'Jet', 'selectedElectron_1', 'myElectron', 'selectedElectron_2', 'selectedElectron_3', 'selectedElectron_f']
['charge', 'dxy', 'dxyErr', 'dz', 'dzErr', 'eta', 'flag', 'mass', 'phi', 'pt', 'invpt']
['Electron', 'Muon', 'Jet', 'selectedElectron_1', 'myElectron', 'selectedElectron_2', 'selectedElectron_3', 'selectedElectron_f']
['charge', 'dxy', 'dxyErr', 'dz', 'dzErr', 'eta', 'flag', 'mass', 'phi', 'pt', 'invpt']
['Electron', 'Muon', 'Jet', 'selectedElectron_1', 'myElectron', 'selectedElectron_2', 'selectedElectron_3', 'selectedElectron_f']
['charge', 'dxy', 'dxyErr', 'dz', 'dzErr', 'eta', 'flag', 'mass', 'phi', 'pt', 'invpt']


## Summary of basics

So to put this together into a single code block, suppose our analysis consisten of selecting events that have at least 2 electron with $p_{T} > 50GeV$, $|\eta| < 0.5$, and we want to calculate the average of all such electron's iverserse $p_{T}$ within the selected events. Our awkward array code would look something like:


In [17]:
events = NanoEventsFactory.from_root( 'file:dummy_nanoevents.root',
                                      'Events',                      
                                      entry_stop=50,                 
                                      schemaclass=DummySchema).events() 

## Object selection 
selectedElectron = events.Electron[ (events.Electron.pt > 50) & 
                                    (np.abs(events.Electron.eta)<0.5) ]
selectedElectron['invpt'] = 1/selectedElectron.pt
events['selectedElectron'] = selectedElectron

# Event selection 
events = events[ak.count(events.selectedElectron.pt,axis=-1) >= 2]

# Calculating the total average 
print(ak.sum(events.selectedElectron.invpt)/ak.count(events.selectedElectron.invpt))



0.01744755860921499


On total this is 4 statements (not counting the file reading step) used to make this analysis. Compare that with the loop based notation:

In [18]:
events = NanoEventsFactory.from_root( 'file:dummy_nanoevents.root',
                                      'Events',                      
                                      entry_stop=50,                 
                                      schemaclass=DummySchema).events() 

count = 0 
suminv = 0 
for i in range(len(events)):
    is_good = [] 
    for j in range(len(events[i].Electron)):
        if events[i].Electron[j].pt > 50 and np.abs(events[i].Electron[j].eta) < 0.5:
            is_good.append(j)
    if len(is_good) >= 2:
        for j in is_good:
            count = count +1 
            suminv += 1.0/ events[i].Electron[j].pt

print(suminv/count)

0.017447559494651994


Notice the results are only difference because the 32bit to 64 bit float conversion is happening at different places. For awkward arrays, this is happening only after the sum has been performed. For the loop based approach this happening everytime the `+=`  operator is called.

For the loop based analysis, notice for such a simple analysis, many many lines of code are dedicated to just book keeping stuff: number of electrons passing criteria, adding a counter variable and sum variable... etc, instead of actualy analysis computation. The array based notation for expressing the analysis is much cleaner, if rather more unfamiliar to typical users. 

Of course, this isn't the end. Physics analysis are typically more involved that just basic selection and counting. In the next session, we will talk about how to perform more involed calculations with awkward arrays that involves multiple collections within an event collection.